### Paras' Approach

In [ ]:
from pyxcel.sheet_manager import SheetManager

In [ ]:
from tqdm.notebook import trange,tqdm

In [ ]:
import numpy as np
import itertools
import pandas as pd
import seaborn as sns

In [ ]:
workbook=SheetManager('Design_Spreadsheet.xlsx')

In [ ]:
input_keys=['Aerofoil','AR','Taper Ratio','b','h']
input_locations=[('Aerodynamics','V2'),('Aerodynamics','C3'),('Aerodynamics','C7'),('Structures','C2'),('Structures','C3')]
input_cell_locations = dict(zip(input_keys,input_locations))
input_cell_locations

In [ ]:
aerofoils = workbook.get_cell('Aerofoil Data','A1:A50').value
aerofoils= list(filter(None, aerofoils))

ARs = np.arange(4,12,3)
taper_ratios = np.arange(0.25,0.61,0.2)
 
widths = np.linspace(4,10,3)*1e-2
heights = np.linspace(4,10,3)*1e-2

In [ ]:
output_keys=['Battery Mass','Max Stress']
output_locations=[('Mass Breakdown','C32'),('Structures','C4')]
output_cell_locations = dict(zip(output_keys,output_locations))
output_cell_locations

In [ ]:
combinations=list(itertools.product(aerofoils,ARs,taper_ratios,widths,heights))
df=pd.DataFrame(combinations,columns=input_keys)
for key in output_keys:
    df[key]=None
    

In [ ]:
df

In [ ]:
for i in trange(len(df.index)):
    for key in input_keys:
        workbook.change_cell(*input_cell_locations[key],df[key][i])
    for key in output_keys:
        df[key][i]=workbook.check_cell(*output_cell_locations[key])

In [ ]:
df

### Dan's Approach

**Imports:**

In [35]:
import os
import csv
import enum
import tqdm
import numpy as np
import itertools
import datetime
from collections import namedtuple, OrderedDict

from pyxcel.sheet_manager import SheetManager
from utils.check_spar import check_spar

**DS Definitions:**

In [47]:
class SheetReference(enum.Enum):
    
    # inputs
    Aerofoil = ('Aerodynamics', 'V2')
    AR = ('Aerodynamics', 'C3')
    TaperRatio = ('Aerodynamics', 'C7')
    
    # outputs
    BatteryMass = ('Mass Breakdown', 'C32')
    MaxStress = ('Structures', 'L78')
    
    # other
    AerofoilRange = ('Aerofoil Data','A1:A50')
    Chord = ('Stability', 'K12')
    SparWidth = ('Structures', 'C2')
    SparHeight = ('Structures', 'C3')

In [48]:
class CombinationReference(enum.IntEnum):
    
    Aerofoil = 0
    AR = 1
    TaperRatio = 2

In [49]:
SparDesign = namedtuple('SparDesign', ('h', 'b', 'ix'))

**Problem Setup:**

In [50]:
# set tolerance for spar height / width - 0.1 denotes 10%
hb_tolerance = 0.1

In [62]:
sheet_path = 'tmp_designsheet.xlsx'
af_dir = os.path.join(os.getcwd(), 'aerofoil_dat')

In [63]:
workbook = SheetManager(sheet_path)

In [64]:
input_keys = (SheetReference.Aerofoil, SheetReference.AR, SheetReference.TaperRatio)
output_keys = (SheetReference.BatteryMass, SheetReference.MaxStress)

In [65]:
# check all input keys are included in CombinationReference
assert set(ref.name for ref in CombinationReference) == set(ref.name for ref in input_keys)

# check all input/output keys are defined
assert all(SheetReference[key.name] is not None for key in itertools.chain(input_keys, output_keys))

In [66]:
aerofoils = list(filter(None, workbook.get_cell(*SheetReference.AerofoilRange.value).value))

# need to increase resolution here
ARs = np.arange(4, 12, 3)
taper_ratios = np.arange(0.25, 0.61, 0.2)

In [67]:
design_combinations = list(itertools.product(aerofoils, ARs, taper_ratios))

**Run Design Sweep:**

In [68]:
# define t_epoch to avoid overwriting file
t_epoch = int(datetime.datetime.now().timestamp())
csv_path = f'design_sweep_{t_epoch}.csv'

for idx, combination in enumerate(tqdm.tqdm(design_combinations)):
    
    # setup dict
    design_params = OrderedDict()
    
    # change each cell
    for key in input_keys:
        workbook.change_cell(*key.value, combination[CombinationReference[key.name]])
        design_params[key.name] = combination[CombinationReference[key.name]]
        
    # load aerofoil
    af_path = os.path.join(af_dir, f'{combination[CombinationReference.Aerofoil]}.dat')
    
    try:
        af = np.loadtxt(af_path, skiprows=1, delimiter=',')
    except OSError:
        continue
    
    # get aerofoil chord length
    chord_length = workbook.check_cell(*SheetReference.Chord.value)
    
    b_range = np.linspace(1e-2, 0.9 * chord_length, 10)
    h_range = np.linspace(1e-2, chord_length / 2, 10)
    
    # get spar with max(ix)
    spar = SparDesign(0, 0, 0)
    for b, h in itertools.product(b_range, h_range):
        ix = b * h ** 3 / 12
        
        if check_spar(af, chord_length, h, b, h * hb_tolerance, b * hb_tolerance) and ix > spar.ix:
            spar = SparDesign(h, b, ix)
            
    # set spar values
    workbook.change_cell(*SheetReference.SparHeight.value, spar.h)
    design_params[SheetReference.SparHeight.name] = spar.h
    
    workbook.change_cell(*SheetReference.SparWidth.value, spar.b)
    design_params[SheetReference.SparWidth.name] = spar.b
    
    for key in output_keys:
        design_params[key.name] = workbook.check_cell(*key.value)
        
    print(design_params)
        
    # write to csv
    csv_exists = os.path.exists(csv_path)
    with open(csv_path, 'a+') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=design_params, delimiter=',', lineterminator='\n')
        
        if not csv_exists:
            writer.writeheader()
            
        writer.writerow(design_params)

  3%|▎         | 6/180 [00:01<00:45,  3.86it/s]


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [112]:

csv_exists = os.path.exists('tmp_test.csv')
with open('tmp_test.csv', 'a+') as csvfile:
    fieldnames = list(tmp.keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    if not csv_exists:
        print("HERE")
        writer.writeheader()
        
    writer.writerow(tmp)
    

In [82]:
fieldnames

['a', 'b', 'c']